In [1]:
# default_exp deps
# hide
_FNAME='deps'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [9]:
#export 
import os
import sys
import logging
logger = logging.getLogger()
import collections
import argparse

from dvcrecord.utils import maybe_yaml

In [3]:
#export

DVC_RECORD_STRING = '--dvc_record'
DVC_DRY_RUN = '--dvc_dryrun'
DO_NOT_INCLUDE_IN_PIPELINE = [DVC_RECORD_STRING, DVC_DRY_RUN]

def add_dvcr_commands(parser):
    parser.add_argument(DVC_RECORD_STRING, action='store_true', dest='dvc_record', 
                        help='Include this flag to record the dvc params entries')
    parser.add_argument(DVC_DRY_RUN, action='store_true', dest='dvc_dryrun', 
                        help='Include this flag to not actually write the dvc.yaml but just display the params that would be written')
    parser.add_argument('--dvc_dep', action='append', nargs='?', 
                        help='Filepaths to the dependency inputs')
    return parser

def make_parser(parser=None):
    parser = parser or argparse.ArgumentParser(description='Stock parser for dvcrecord commands')    
    parser = add_dvcr_commands(parser)
    return parser

In [4]:
def test_parser():
    parser = make_parser()
    #add custom parser code
    parser.add_argument("--athing", type=int)
    ns = parser.parse_args(['--athing=10', '--dvc_record'])
    TESTCASE.assertTrue(ns.dvc_record)
    TESTCASE.assertEqual(ns.athing, 10)
    
    ns2 = parser.parse_args([])
    TESTCASE.assertFalse(ns2.dvc_record)
test_parser()

In [5]:
def test_dependency_cli():
    parser = make_parser()
    ns = parser.parse_args(['--dvc_dep', 'first.txt' ,'--dvc_dep', 'second.txt'])
    TESTCASE.assertEqual(ns.dvc_dep, ['first.txt', 'second.txt'])

test_dependency_cli()

In [11]:
#export

class NoArgumentNamed(Exception):
    pass

class Dependency:
    def __init__(self, namespace=None):
        self.namespace = namespace
        self.deps = []
        
    def register(self, *args):
        fpath = os.path.join(*args)
        if fpath not in self.deps:
            self.deps.append(fpath)
        return fpath
        
    def from_cli(self, namespace=None, arg_name='dvc_dep'):
        namespace = namespace or self.namespace
        if namespace:
            try:
                return getattr(namespace, arg_name)
            except AttributeError:
                logger.info("Looked for dvc file dependencies in an argument named '{}' but it wasnt in the command line".format(arg_name))
                return []
        else:
            return []
        
    #def register_param(self, param):
    #    for fpath in param.list_of_files():
    #        self.register(fpath)
            
    def register_sourcecode(self, command=None):
        command = command or sys.argv[:]
        self.register(command[0])        
            
    def render(self, as_yaml=False, *args, **kwargs):
        returnme = self.deps[:]
        
        deps_in_cli = self.from_cli(*args, **kwargs)
        if deps_in_cli:
            returnme.extend(deps_in_cli)
        return maybe_yaml(returnme, as_yaml=as_yaml)
        

In [7]:
import argparse
from tempfile import TemporaryDirectory

from dvcrecord.utils import write_yaml


def test_dep():    
    parser = make_parser()
    ns = parser.parse_args(['--dvc_dep', 'first.txt' ,'--dvc_dep', 'second.txt'])
    dep = Dependency(namespace=ns)    
    TESTCASE.assertEqual(dep.from_cli(), ['first.txt', 'second.txt'])

    ns = argparse.Namespace()
    TESTCASE.assertEqual(dep.from_cli(ns), [])
        
    with TemporaryDirectory() as tempdir:
        depfile = dep.register(tempdir, 'testdep.yaml')
        write_yaml({'try':'me'}, tempdir, 'testdep.yaml')
        with open(depfile) as f:
            f.read()
        TESTCASE.assertEqual(dep.render(as_yaml=False), 
                             [depfile, 'first.txt', 'second.txt']
                            )        
test_dep()

In [10]:
notebook2script(_nbpath)

Converted deps.ipynb.
